In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [12]:
from google.colab import files
files.upload()  # upload kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yaswanthkancharla","key":"743b06db12fc339b072c8230b4a5e855"}'}

In [13]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d kutaykutlu/drowsiness-detection

Dataset URL: https://www.kaggle.com/datasets/kutaykutlu/drowsiness-detection
License(s): unknown


In [15]:
!unzip drowsiness-detection.zip -d drowsiness_dataset

Streaming output truncated to the last 5000 lines.
  inflating: drowsiness_dataset/open_eye/s0024_00466_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00467_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00468_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00469_0_1_1_2_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00470_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00471_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00472_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00473_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00474_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00475_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00476_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00477_0_1_1_0_0_01.png  
  inflating: drowsiness_dataset/open_eye/s0024_00478_0_1_1_0_0_01.png  
  inflating: 

In [17]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

dataset_path = "drowsiness_dataset"
categories = ["closed_eye", "open_eye"]  # match the extracted folder names exactly

data = []
labels = []

img_size = 64  # Resize all images to 64x64

# Load and preprocess
for category in categories:
    path = os.path.join(dataset_path, category)
    class_num = categories.index(category)

    for img_name in os.listdir(path):
        try:
            img_path = os.path.join(path, img_name)
            img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_array = cv2.resize(img_array, (img_size, img_size))
            data.append(img_array)
            labels.append(class_num)
        except Exception as e:
            print("Error loading image:", e)

# Convert and normalize
data = np.array(data).reshape(-1, img_size, img_size, 1) / 255.0
labels = np.array(labels)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels)

# A1

In [24]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 682,753 (2.60 MB)

 Trainable params: 682,753 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 150s 124ms/step - accuracy: 0.8775 - loss: 0.2726 - val_accuracy: 0.9649 - val_loss: 0.1012
Epoch 2/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 199s 121ms/step - accuracy: 0.9779 - loss: 0.0655 - val_accuracy: 0.9877 - val_loss: 0.0347
Epoch 3/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 204s 123ms/step - accuracy: 0.9861 - loss: 0.0447 - val_accuracy: 0.9885 - val_loss: 0.0326
Epoch 4/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 199s 121ms/step - accuracy: 0.9889 - loss: 0.0351 - val_accuracy: 0.9854 - val_loss: 0.0416
Epoch 5/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 202s 121ms/step - accuracy: 0.9901 - loss: 0.0296 - val_accuracy: 0.9920 - val_loss: 0.0230
Epoch 6/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 203s 122ms/step - accuracy: 0.9919 - loss: 0.0253 - val_accuracy: 0.9909 - val_loss: 0.0286
Epoch 7/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 201s 121ms/step - accuracy: 0.9929 - loss: 0.0222 - val_accuracy: 0.9928 - val_loss: 0.0212
Epoch 8/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 202s 120ms/step - ac

In [20]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.9937 - loss: 0.0269
Test Accuracy: 0.9936


# A2

In [25]:
_ = model.predict(X_train[:1])  # Ensures model is built

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


In [26]:
from tensorflow.keras import Model

# Rebuild feature extractor using layer tensors (not .input attribute)
feature_extractor = Model(
    inputs=model.layers[0].input,
    outputs=model.layers[-2].output  # second last layer
)

In [27]:
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

print("Feature shape:", X_train_features.shape)

1200/1200 ━━━━━━━━━━━━━━━━━━━━ 40s 33ms/step
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step
Feature shape: (38400, 128)


In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_features, y_train)

y_pred_knn = knn.predict(X_test_features)
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")


KNN Accuracy: 0.9832


In [29]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_features, y_train)

y_pred_rf = rf.predict(X_test_features)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")


Random Forest Accuracy: 0.9777


In [30]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train_features, y_train)

y_pred_svm = svm.predict(X_test_features)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")


SVM Accuracy: 0.8747


In [31]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_features, y_train)

y_pred_nb = nb.predict(X_test_features)
print(f"Naïve Bayes Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}")


Naïve Bayes Accuracy: 0.6089


# **Regression Models**

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train_features, y_train)

y_pred_lr = lr.predict(X_test_features)
print(f"Linear Regression MSE: {mean_squared_error(y_test, y_pred_lr):.4f}")


Linear Regression MSE: 0.1071


In [33]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X_train_features, y_train)

y_pred_lasso = lasso.predict(X_test_features)
print(f"LASSO Regression MSE: {mean_squared_error(y_test, y_pred_lasso):.4f}")


LASSO Regression MSE: 0.2500


In [34]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf')
svr.fit(X_train_features, y_train)

y_pred_svr = svr.predict(X_test_features)
print(f"SVR MSE: {mean_squared_error(y_test, y_pred_svr):.4f}")


SVR MSE: 0.0680
